# Using TensorFlow to create and train neural networks

To work with TensorFlow, we first need to install it. Uncomment the code below, if you haven't already done so (note that we use `pydot` and `graphviz` for visualizing the hidden layers of a neural network - but they are not necessary to run neural networks in TensorFlow).

Note that parts 4 and 6 of the notebook build on a GitHub repository by [Aurélien Géron](https://github.com/ageron/handson-ml2).

In [ ]:
#pip install tensorflow
#pip install pydot
#pip install graphviz

Once we have installed all packages, let's do the obligatory imports:

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# 1. Getting started

## 1.1 Creating tensors

There are two types of "variables" in TensorFlow that you might work with. Actual variables, `tf.Variables`, which can be changed once they have been set, as well as constants , `tf.constant`, which cannot be changed anymore. Let's create a random array of data `X`, as well as parameter matrices `W` and `b`. Can you see why we would use `tf.constant` for `X` and `tf.Variable` for `W` and `b`?

In [ ]:
X = tf.constant(np.random.randn(100,5), name = "X")
X.shape

Note that the "tensor" we create has exactly the dimension of the numpy array we use to initialize it. TensorFlow allows you to use `numpy` data quite seamlessly.

In [ ]:
W = tf.Variable(np.random.randn(5,3), name = "W")
W.shape

In [ ]:
b = tf.Variable(np.random.randn(1,3), name = "b")
b.shape

## 1.2 Basic operations on tensors

Let's now perform some basic operations on tensors. These work essentially the same way as in `numpy` (e.g., the typical rules of matrix-multiplication apply when we use `tf.matmul`), just that we have to learn a few different key-words:

In [ ]:
z = tf.add(tf.matmul(X,W),b)
z.shape

TensorFlow has all the standard activation functions in-built, for example, the logistic sigmoid one:

In [ ]:
a = tf.keras.activations.sigmoid(z)
a.shape

## 1.3 Type-casting

One aspect of TensorFlow you might know from other programming languages, but which is usually taken care of for you in base Python, is that types are not automatically "cast". What that means is that we cannot run operations such as multiplying an integer and a float tensor. We cannot even multiply a `float32` tensor with a `float64` tensor! You can find the datatype of any tensor by printing it out, or by specifically looking at `tensor.dtype`:

In [ ]:
b

In [ ]:
b.dtype

We will now (explicitly) convert `b` from a `tf.float64` data-type (a float with 64 bits of precision), to a `tf.float32` data-type (a float with 32 bits of precision):

In [ ]:
b = tf.cast(b,tf.float32)

Note that `X` and `W` are still of type `tf.float64` (the default). Let's try the following:

In [ ]:
tf.add(tf.matmul(X,W),b)

This didn't work! Let's re-cast `b` into a `tf.float64`

In [ ]:
b = tf.cast(b, tf.float64)

In [ ]:
b.dtype

In [ ]:
tf.add(tf.matmul(X,W),b)

Now it works again!

## 1.4 Some other useful operations

Let's look at `tf.one_hot` and `tf.reshape`, two operations you come across a lot when handling data in TensorFlow. Can you see what they do?

In [ ]:
y = tf.constant(3)
y_one_hot = tf.one_hot(y,4,axis=0)
y_one_hot

In [ ]:
y = [2]
y_one_hot = tf.one_hot(y,4,axis=0)
y_one_hot

In [ ]:
tf.reshape(y_one_hot,(4,))

# 2. Creating a simple neural network with the Sequential API

We will recreate what we did in the last lecture: a logistic regression model that allows us to differentiate between sign-language digits 0 and 1. But this time, we will be using TensorFlow's Sequential API to build a (single-neuron) neural network. The data-handling is the same as last week:

In [ ]:
X = np.load('digits_X.npy')
y = np.load('digits_y.npy')
X = np.concatenate((X[204:409], X[822:1028] ), axis=0)
z = np.zeros(409-204)
o = np.ones(1028-822)
y = np.concatenate((z, o), axis=0).reshape(X.shape[0],1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=172)

## 2.1 Building the model

Let's create the model. Notice that we haven't flattened the observations (recall that each value of `X` is a matrix!). The beauty about TensorFlow is that we can simply add in a `Flatten` layer, to do this for us. Hence, we have two layers in our model: the `Flatten` layer (not an actual neural network layer), and a standard `Dense` layer, with one neuron/unit (the one that performs the logistic regression).

For our `Dense` layer with one neuron, we also need to specify the activation function. Of course, for a logistic regression, we use the logistic sigmoid function, simply `'sigmoid'` in TensorFlow.

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(64, 64)),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

Now that we have created our model, we can take a look at its layers:

In [ ]:
model.layers

The `model.summary()` is a bit more informative:

In [ ]:
model.summary()

The `Flatten` layer flattens each input from a (64,64)-matrix to a (4096,)-vector. However, it says (None, 4096) for the output shape. Can you guess why?

Also, can you guess what the 4097 parameters are?

In order to use our model, we also have to `compile` it. This is the step where we decide on
- the optimization algorithm (we will use `'sgd'`, or "stochastic gradient descent", which is very similar to the gradient descent method we've seen before - we will deal with the exact difference later),
- the loss function (we will use `'binary_crossentropy'`, as before),
- the metrics to keep track of as the model is being trained, in the form of a list (we will only keep track of `'accuracy'` for now).

In [ ]:
model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

That's it! When we compile the model, it automatically creates the forward propagation and back-propagation steps for us, based on the layers and cost function we define!

## 2.2 Training the model

We will now train the model, using the training data. We will run it for 100 "epochs", that is iterations of our optimization algorithm:

In [ ]:
model.fit(X_train, y_train, epochs=100)

Let's restart the process (we will use `clear_session` to properly clear out TensorFlow's cache, and then define random seeds for `numpy` and `tf` (this is good practice to do right after the clear out). Other than that, we will only make three small changes:
- We add validation data to the training process (we will here simply use the test data - not best practice, but this is just for showing you how the TensorFlow training can directly consider a validation data set)
- Instead of using the default `'sgd'` optimizer, we manually specify the learning rate, using the `tf.keras.optimizers.SGD` object
- We also save the output of `model.fit`, as this will be interesting for drawing graphs

In [ ]:
tf.keras.backend.clear_session()
np.random.seed(672)
tf.random.set_seed(323)

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(64, 64)),
  tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
              loss='binary_crossentropy',
              metrics=['accuracy'])
log = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

We can now use the log to see how the training went. The `log.history` is a dictionary that will capture, for each epoch, all the metrics we defined for our model (`['accuracy']` in our case), as well as the cost/loss, for both the training and the "validation" dataset. The keywords to use in the dictionary are simply the words you can find in the log above.

In [ ]:
plt.plot(log.history['accuracy'],label = "training accuracy")
plt.plot(log.history['val_accuracy'], label = "testing accuracy")
plt.legend()
plt.show()

We can also look at the parameters and how they have been adjusted when the model was trained. Remember that `model.layers` gives us all the layers of our model:

In [ ]:
model.layers

We can access these layers with standard Pythonic indexing. E.g., the output layer in our case is layer 1:

In [ ]:
output_layer = model.layers[1]

Let's now take a look at both the weights and the bias after training. Before you run the cell, ask yourself: how many weights and biases do you expect there to be?

In [ ]:
weights, bias = output_layer.get_weights()
print(weights)
print('----')
print(weights.shape)
print('----')
print(bias)

## 2.3 Making predictions

To make predictions, we simply input the data we want to make predictions on into the model we have trained. `.numpy()` just helps with formatting

In [ ]:
predictions = model(X_test).numpy()

As before, our predictions are values between 0 and 1. let's get actual binary classification predictions:

In [ ]:
print(predictions > 0.5)

# 3. Creating a deeper neural network with TensorFlow

We will now create a deeper neural network with TensorFlow's Sequential API (the same one we were just using now). Before we do so, let's properly clear out the cache and set some new random seeds:

In [ ]:
tf.keras.backend.clear_session()
np.random.seed(745)
tf.random.set_seed(498)

We can now create the model. We can either create it in one go:
```
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(64, 64)),
  tf.keras.layers.Dense(3, kernel_initializer = 'uniform', activation='relu')
  tf.keras.layers.Dense(3, kernel_initializer = 'uniform', activation='relu')
  tf.keras.layers.Dense(1, kernel_initializer = 'uniform', activation='sigmoid')
])
```
or we can add layers one-by-one:

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(64,64)))
model.add(tf.keras.layers.Dense(3, kernel_initializer = 'uniform', activation="relu"))
model.add(tf.keras.layers.Dense(3, kernel_initializer = 'uniform', activation="relu"))
model.add(tf.keras.layers.Dense(1, kernel_initializer = 'uniform', activation="sigmoid"))

A few things to note: We are using the `Flatten` layer again, to convert our input-matrices into input-vectors. We then add a number of hidden layers and an output layer. Can you see how many hidden layers there are and how many neurons are at each?

As in the tutorial, we use `'relu'` units at the hidden layer and a `sigmoid` unit at the output layer. We also add in a `kernel_initializer`: we are specifying how the parameters should be initialized (uniformly random). In this particular case, the default initialization is likely not to work, but feel free to try it out as well.

In general, if you are unsure about any of the inputs to the TensorFlow functions, make sure to check the [amazing documentation](https://www.tensorflow.org/api_docs/python/tf/all_symbols).

As before, we can print out a summary of the `model`:

In [ ]:
model.summary()

Do the shapes make sense to you?

Can you account for the number of parameters?

We can also visualize the layers of our model:

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

And as before, we can take a look at it's parameters (note that we haven't trained the model yet, so these are the randomly generated weights and biases):

In [ ]:
hidden_layer = model.layers[1]

In [ ]:
weights, bias = hidden_layer.get_weights()
print(weights)
print('----')
print(weights.shape)
print('----')
print(bias)
print('----')
print(bias.shape)

Finally, before training the model, we need to compile it again. As before, we use `'sgd'` as an optimization algorithm, and `'binary_crossentropy'` as our loss (or cost) function:

In [ ]:
model.compile(loss="binary_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
              metrics=["accuracy"])

## 3.1 Training the model

Let's now run the model for 100 epochs, similar to what we did before:

In [ ]:
log = model.fit(X_train, y_train, epochs=100,validation_data=(X_test, y_test))

Did you see anything happening? Why do you think this is the case? What do you think happens if we really crank up the iterations (say, to a few thousand)?



We will try again, this time with a different optimization-algorithm, `'adam'` (again, don't worry too much about what exactly `'adam'` does, we will see this later in class). For this, we have to recompile the model. Keep in mind, however, that the weights and biases will already be at exactly the level they have been trained to in the previous training attempt:

In [ ]:
model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=["accuracy"])

Let's try again:

In [ ]:
log = model.fit(X_train, y_train, epochs=100,validation_data=(X_test, y_test))

Looks much better, right? As before, we can display the training process:

In [ ]:
plt.plot(log.history['accuracy'],label = "training accuracy")
plt.plot(log.history['val_accuracy'], label = "testing accuracy")
plt.legend()
plt.show()

# 4. Can you build on what you have seen? Creating a neural network for regression

We will now use a neural network for regression instead of classification. In particular, we will load a standard scikit-learn dataset that contains key characteristics of some houses in california, as well as their values (in $100,000). We will then try to predict the houses' values.

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

In [ ]:
housing = fetch_california_housing()

The housing data comes as a dictionary: We can find the features of the data as follows:

In [ ]:
housing['feature_names']

The actual observations (or, more precisely, the feature matrix `X`) comes with the key `'data'`:

In [ ]:
housing['data'].shape

The labels come with the key `'target'`:

In [ ]:
housing['target'].shape

As always, we start by splitting the data. We will use a training, a validation, and a testing set:

In [ ]:
X_train, X_other, y_train, y_other = train_test_split(housing['data'], housing['target'], train_size=0.7, random_state=461)
X_test, X_valid, y_test, y_valid = train_test_split(X_other, y_other, train_size = 0.5, random_state=391)
print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)

Next, we normalize the X-data. Keep in mind to not use the validation or test data in defining how the scaling process works (otherwise, you can leak information about this data into your training process). You can, however, use the same scaler on validation and test data, rather than defining your own scalers here - this will usually work better than normalizing all data independently:

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

## 4.1 Building a neural network

Let's clear our TensorFlow session and set new seeds. Even if not strictly necessary, this is good practice.

In [ ]:
tf.keras.backend.clear_session()
np.random.seed(231)
tf.random.set_seed(631)

Now it's your turn! Build a neural network with three `Dense` layers: two hidden layers with `'relu'` activation, and one final output layer. The hidden layers should have 10 neurons each. For the first hidden layer, you have to specify the correct `input_shape`. Recall that this corresponds to the columns in our X-data. But make sure that `input_shape` is a list or a tuple, not just a single value, otherwise there will be an error.

For the output layer, consider whether we need any activation function at all. Remember that a linear regression is just a weighted sum of inputs plus a bias (sounds familiar?)

Save your model as `model`.

If you are stuck, first take a look at the [documentation](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense) of the `Dense` layers.

As always, it helps to visualize your model with the `summary()` function:

As before, try to answer the following questions:
- Do the shapes make sense to you?
- Can you account for the number of parameters?

Now, compile your model. We are doing regression, so it is quite natural to use `'mean_squared_error'` as the loss function.

Moreover, we will us `'sgd'` as before. However, instead of specifying `optimizer = 'sgd'`, we will make a custom optimizer function, which allows us to choose our own learning rate. In particular, you should use
```
optimizer=tf.keras.optimizers.SGD(learning_rate=0.03))
```

Finally, since we are not doing classification, there is no accuracy to keep track of, so simply leave out the `metrics` argument.

Train your model for 30 epochs, also keeping track of the `validation_data`. Save the output as `log`.

Use the code below to display the loss on both the training and the validation set:

In [ ]:
plt.plot(log.history['loss'],label = "training loss")
plt.plot(log.history['val_loss'], label = "validation loss")
plt.legend()
plt.show()

We can evaluate the model on the test set directly, using `model.evaluate(X_test,y_test)`. Try it out:

Finally, we can take a look at some examples and see how well we are doing on those. Run the code below, and feel free to adjust the examples.

In [ ]:
X_new = X_test[:5]
y_predict = model.predict(X_new)
y_predict = y_predict.reshape(-1,)
y_predict = [round(y,3) for y in y_predict]

In [ ]:
print("Predicted values: " + str(y_predict))
print("True values: " + str(y_test[:5]))

Can you do better? Try tweaking the learning rate, the number of layers, and the neurons per layer to see if your validation loss improves. Once you have decided on your final model, evaluate it on the test set and note down your loss there.

## 4.2 Visualization with TensorBoard

We will now see how we can use TensorBoard to get more in-depth visualizations of our training process and our model. Let's start by cleaning up and making some additional imports:

In [ ]:
tf.keras.backend.clear_session()
np.random.seed(231)
tf.random.set_seed(631)

In [ ]:
import os
import datetime

We can now load TensorBoard into the Jupyter Notebook and specify where the relevant logs should be kept that are needed to visualize the training process:

In [ ]:
%load_ext tensorboard
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

We build the same baseline model as before:

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, activation="relu", input_shape=X_train.shape[1:]),
    tf.keras.layers.Dense(10, activation="relu"),
    tf.keras.layers.Dense(1)
])
model.compile(loss="mean_squared_error",
              optimizer=tf.keras.optimizers.SGD(learning_rate=0.03))

We train our model as before, but we add a so-called `callback`. There are many types of callbacks, and you will discover some of them later, but TensorBoard brings its very own one.

Note that we are defining a list of `callbacks`, but for now, we keep it at a single one:

In [ ]:
tensorboard_cb = tf.keras.callbacks.TensorBoard(logdir)
history = model.fit(X_train, y_train, epochs=30,
                    validation_data=(X_valid, y_valid),
                    callbacks=[tensorboard_cb])

In the training process, nothing has chaged, except that you will now find a log in your working directory. We can call upon that log to display the TensorBoard as follows (if you get an error, keep reading below)

In [ ]:
%tensorboard --logdir logs

You may run into an issue here because of a conflict between TensorBoard and the environment you are using. If this is the case, you need to specify exactly where TensorBoard can be found. It should be within your environment's folder, under "bin". You can find an example below. Uncomment the next lines and adjust your path accordingly:

In [ ]:
# os.environ['TENSORBOARD_BINARY'] = '/Users/philippe/anaconda3/envs/adl_env/bin/tensorboard'
# %tensorboard --logdir logs

Enjoy exploring the TensorBoard. We will see later that we can still add a lot more functionality to it. It is particularly nice for comparing different models.

## 4.3 Saving and loading models

Sometimes, we need to save our models, instead of training them from scratch (training deep neural networks can take quite some time!) TensorFlow enables this quite easily, for example with the following code. Once you have saved the model, check your working directory and you will find it there (as an "h5" file).

In [ ]:
model.save("my_tf_regression_model.h5")

We can now call up the (trained) model, just as easily:

In [ ]:
saved_model = tf.keras.models.load_model("my_tf_regression_model.h5")

Verify that this is the trained model by running the predictions on it. You should get the same result as with the original model:

In [ ]:
X_new = X_test[:5]
y_predict = saved_model.predict(X_new)
y_predict = y_predict.reshape(-1,)
y_predict = [round(y,3) for y in y_predict]
print("Predicted values: " + str(y_predict))
print("True values: " + str(y_test[:5]))

Sometimes, you might want to save only the parameters of the model, not the entire model. Again, TensorFlow allows us to save and load the parameters quite easily:

In [ ]:
model.save_weights("my_tf_regression_weights.ckpt")

In [ ]:
model.load_weights("my_tf_regression_weights.ckpt")

## 4.4 Callbacks

When using TensorBoard, we already saw a Callback in action. We will now get back to those, more generally. The idea is, that you might want to "do stuff" while training your model (for example, saving your current progress, in case your computer crashes). We start by defining our model:

In [ ]:
tf.keras.backend.clear_session()
np.random.seed(231)
tf.random.set_seed(631)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, activation="relu", input_shape=X_train.shape[1:]),
    tf.keras.layers.Dense(10, activation="relu"),
    tf.keras.layers.Dense(1)
])
model.compile(loss="mean_squared_error",
              optimizer=tf.keras.optimizers.SGD(learning_rate=0.03))

Let's now create a `ModelCheckpoint`, which is a call-back that saves your model in each iteration (epoch) of training. That is, whatever you might do later on with your model, you can always get back to the trained version.

In your checkpoint, you will need to specify how the model should be saved as. Moreover, if you are using `validation_data` when training your model, you can also specify `save_best_only=True`. This means, that the callback looks at your validation_loss, and only saves the model at its current state if the validation loss is better (i.e. lower) than the one in the previous saved version. Quite useful, right?

In [ ]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("my_best_tf_regression_model.h5", save_best_only=True)

Once we have generated the callback, we add it to our callback list in training, as we did for TensorBoard. Nothing else changes:

In [ ]:
history = model.fit(X_train, y_train, epochs=30,
                    validation_data=(X_valid, y_valid),
                    callbacks=[checkpoint_cb])

You can now run `tf.keras.models.load_model("my_best_tf_regression_model.h5")` to call up the version of your model that had the best validation loss.

Another useful callback to try out (this is left to you) is the `EarlyStopping` callback, which stops the training process if the validation loss isn't improving anymore. You can call it as follows:
```
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
```

Here, `patience` defines the number of epochs which the callback waits for improvements before stopping the training. `restore_best_weights=True` means that, once training is stopped, you get the version of the model that led to the best validation loss, not the final version of the model.

# 5. Multi-class classification with softmax

We will now see how to use a softmax activation function to classify observations into multiple classes. As always, we start by cleaning up:

In [ ]:
tf.keras.backend.clear_session()
np.random.seed(583)
tf.random.set_seed(21)

For this example, we will use the full digits dataset (recall that we had images of ten sign-language digits, with the corresponding one-hot-encoded labels). We will split the dataset into training, test and validation sets:

In [ ]:
X = np.load('digits_X.npy')
y = np.load('digits_y.npy')
X_train, X_other, y_train, y_other = train_test_split(X, y, train_size=0.8, random_state=342)
X_test, X_valid, y_test, y_valid = train_test_split(X_other, y_other, train_size = 0.5, random_state=152)
print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)
print(X_test.shape)
print(y_test.shape)

We train a simple feed-forward network, with two hidden layers, plus the softmax layer. Remember also that we need to flatten the image-matrics:

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(64, 64)),
    tf.keras.layers.Dense(300, activation="relu"),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax"),
])

As always, it is useful to take a look at our model and try to spot any possible issues:

In [ ]:
model.summary()

We can then compile the model. For multiple categories, we will usually use the `categorical_crossentropy` loss (with some exceptions, one of which you will come across in this week's homework). We also use the `'adam'` optimization algorithm (with our self-defined learning rate), which we talk about in class next week. Otherwise, let's again keep track of `'accuracy'`:

In [ ]:
model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=["accuracy"])

We can now run the model, we will use 200 epochs for now:

In [ ]:
log = model.fit(X_train, y_train, epochs=100,
                    validation_data=(X_valid, y_valid))

In [ ]:
plt.plot(log.history['accuracy'],label = "training accuracy")
plt.plot(log.history['val_accuracy'], label = "validation accuracy")
plt.legend()
plt.show()

It seems we are experiencing some over-fitting. This is an issue that we will turn to next week!

Finally, let's evaluate the model on the test data:

In [ ]:
model.evaluate(X_test, y_test)

# 6. More multiclass-classification (homework)

For this part, we will train a neural network on a dataset of fashion-products that is labeled with the categories of each product. The data is loaded directly from TensorFlow (which has quite the broad collection of datasets):

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(X_other, y_other), (X_test, y_test) = fashion_mnist.load_data()

In [ ]:
print(X_other.shape)
print(y_other.shape)
print(X_test.shape)
print(y_test.shape)

The X's are matrices (with 28x28 pixels), while the y's are numbers.

We divide the values of X by 255 (essentially standardizing the pixel-values to 0-1) and also split apart a validation set (of the same size as the test set):

In [ ]:
X_other = X_other / 255.
X_test = X_test / 255.
X_train, X_valid, y_train, y_valid = train_test_split(X_other, y_other, train_size = 50000, random_state=152)

print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)

Let's plot two examples:

In [ ]:
plt.subplot(1, 2, 1)
plt.imshow(X_train[0],cmap="binary")
plt.axis('off')
plt.subplot(1, 2, 2)
plt.imshow(X_train[500],cmap="binary")
plt.axis('off')

As well as the corresponding labels:

In [ ]:
print(y_train[0])
print(y_train[500])

That's a bit hard to interpret. Luckily, we have the right names for each label available:

In [ ]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

We can now take another look at what the pictures above represent:

In [ ]:
print(class_names[y_train[0]])
print(class_names[y_train[500]])

## 6.1 Training the model

Let's start with some housekeeping:

In [ ]:
tf.keras.backend.clear_session()
np.random.seed(451)
tf.random.set_seed(346)

Can you create a model with two hidden layers and one (softmax) output layer? The hidden layers should have 100 neurons each. You will have to figure out the number of neurons on the output layer (hint: it depends on the number of classes).

Also, don't forget to flatten the images with an appropriate `input_shape`!

Make sure you save your model as `model`.

Use the `summary` function, to see whether everything worked out as it should. If you defined the model as discussed above, you should get a total of 89,610 parameters.

We can now compile the model. Use `optimizer=tf.keras.optimizers.SGD(learning_rate=0.01)` and `metrics=['accuracy']`. For the loss, use `sparse_categorical_crossentropy`. This is because our y's here are **not** one-hot-encoded, but instead are values from 0 to 9.

Train the model for 30 epochs, keeping track also of the `validation_data`.

Take a look at the training process. The below will only work if you have called your training output `log`:

In [ ]:
plt.plot(log.history['accuracy'],label = "training accuracy")
plt.plot(log.history['val_accuracy'], label = "testing accuracy")
plt.legend()
plt.show()

If we accept our model, we can evaluate it on the test set, using the `evaluate` function of our model:

We can also take a look at some predictions:

In [ ]:
X_new = X_test[:4]
y_predict = np.argmax(model.predict(X_new), axis=-1)
y_predict = [class_names[y] for y in y_predict]

In [ ]:
plt.subplot(1, 4, 1)
plt.imshow(X_test[0],cmap="binary")
plt.axis('off')
plt.subplot(1, 4, 2)
plt.imshow(X_test[1],cmap="binary")
plt.axis('off')
plt.subplot(1, 4, 3)
plt.imshow(X_test[2],cmap="binary")
plt.axis('off')
plt.subplot(1, 4, 4)
plt.imshow(X_test[3],cmap="binary")
plt.axis('off')
print("Predictions are: " + str(y_predict))

Can you do better? Try tweaking the learning rate, the number of layers, and the neurons per layer to see if your validation loss improves. Once you have decided on your final model, evaluate it on the test set and note down your loss there.